In [1]:
import random, time, numpy as np, pandas as pd
from collections import defaultdict
from copy import deepcopy

# ---------------------------
# Parameters (tweak these)
# ---------------------------
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

N = 3           # number of base stations
S = 16          # number of services
U = 100         # number of users

# BS resource ranges
STORAGE_RANGE = (200, 400)   # GB
CPU_RANGE     = (2.0, 6.0)   # GHz
UPLINK_RANGE  = (30, 80)     # Mbps
DOWNLINK_RANGE= (60, 150)    # Mbps

# Service requirement ranges
RS_RANGE = (8, 40)        # GB
CS_RANGE = (0.05, 0.5)    # GHz
BU_RANGE = (0.5, 4)       # Mbps
BD_RANGE = (0.5, 12)      # Mbps

# User coverage
MIN_COV = 1
MAX_COV = 3

# Popularity skew (Zipf)
ZIPF_SKEW = 0.9

# GA hyperparameters
POP_SIZE = 12
GENERATIONS = 12
CROSSOVER_RATE = 0.8
MUTATION_RATE  = 0.04
ELITISM = 2

# ACO hyperparameters
ANTS = 6
ACO_ITERS = 6
ALPHA = 1.0
BETA  = 2.0
RHO   = 0.15

# ---------------------------
# Build synthetic scenario
# ---------------------------
BS = []
for n in range(N):
    BS.append({
        'R': random.randint(*STORAGE_RANGE),
        'C': random.uniform(*CPU_RANGE),
        'Bu': random.randint(*UPLINK_RANGE),
        'Bd': random.randint(*DOWNLINK_RANGE),
    })

ranks = np.arange(1, S + 1)
zipf_weights = 1.0 / (ranks ** ZIPF_SKEW)
zipf_probs   = zipf_weights / zipf_weights.sum()

services = []
for s in range(S):
    services.append({
        'rs': random.randint(*RS_RANGE),
        'cs': random.uniform(*CS_RANGE),
        'bu': random.uniform(*BU_RANGE),
        'bd': random.uniform(*BD_RANGE),
        'pop_prob': float(zipf_probs[s])
    })

users = []
service_choices = list(range(S))
for u in range(U):
    su = int(np.random.choice(service_choices, p=zipf_probs))
    cov_size = random.randint(MIN_COV, min(MAX_COV, N))
    nearby = sorted(random.sample(range(N), cov_size))
    users.append({'s': su, 'nearby': nearby})

users_by_service = defaultdict(list)
for uid, u in enumerate(users):
    users_by_service[u['s']].append(uid)

# ---------------------------
# Helpers
# ---------------------------
def rand_feasible_placement():
    x = np.zeros((N, S), dtype=int)
    for n in range(N):
        perm = list(range(S))
        random.shuffle(perm)
        used = 0
        for s in perm:
            rs = services[s]['rs']
            if used + rs <= BS[n]['R']:
                if random.random() < 0.35:
                    x[n, s] = 1
                    used += rs
    repair_placement(x)
    return x

def placement_to_vector(x):
    return x.flatten()

def vector_to_placement(v):
    return v.reshape((N, S)).astype(int)

def compute_marginal_benefit_per_gb(bs_index, placement):
    benefit = {}
    for s in range(S):
        if placement[bs_index, s] == 0: continue
        count = 0.0
        for uid in users_by_service[s]:
            if bs_index in users[uid]['nearby']:
                other_hosts = [m for m in users[uid]['nearby']
                               if m != bs_index and placement[m, s] == 1]
                count += 1.0 if len(other_hosts)==0 else 0.2
        rs = services[s]['rs']
        benefit[s] = count / (rs + 1e-9)
    return benefit

def repair_placement(placement):
    for n in range(N):
        used = int(placement[n] @ np.array([services[s]['rs'] for s in range(S)]))
        if used <= BS[n]['R']: continue
        benefit = compute_marginal_benefit_per_gb(n, placement)
        placed = [s for s in range(S) if placement[n, s]==1]
        placed_sorted = sorted(placed, key=lambda s: benefit.get(s, 0.0))
        idx = 0
        while used > BS[n]['R'] and idx < len(placed_sorted):
            srm = placed_sorted[idx]
            placement[n, srm] = 0
            used -= services[srm]['rs']
            idx += 1
        if used > BS[n]['R']:
            placed = [s for s in range(S) if placement[n, s]==1]
            random.shuffle(placed)
            for srm in placed:
                placement[n, srm] = 0
                used -= services[srm]['rs']
                if used <= BS[n]['R']:
                    break
    return placement

# ---------------------------
# ACO routing
# ---------------------------
def aco_routing(placement, ants=ANTS, iterations=ACO_ITERS, alpha=ALPHA, beta=BETA, rho=RHO):
    candidates = []
    for u in users:
        su = u['s']
        cand = [n for n in u['nearby'] if placement[n, su]==1]
        candidates.append(cand + [-1] if cand else [-1])
    pher = np.ones((U, N+1)) * (1.0/(N+1))
    best_global, best_global_score = None, -1

    def compute_eta(u_idx, n_idx, remC, remBu, remBd):
        if n_idx == -1: return 0.1
        s = users[u_idx]['s']
        cs, bu, bd = services[s]['cs'], services[s]['bu'], services[s]['bd']
        return max(min(remC[n_idx]/cs, remBu[n_idx]/bu, remBd[n_idx]/bd), 1e-6)

    for _ in range(iterations):
        iter_best_score, iter_best_assign = -1, None
        delta_pher = np.zeros_like(pher)
        for _ in range(ants):
            remC = np.array([BS[n]['C'] for n in range(N)])
            remBu= np.array([BS[n]['Bu'] for n in range(N)])
            remBd= np.array([BS[n]['Bd'] for n in range(N)])
            assignment = [-1]*U; edge_served=0
            for uidx in range(U):
                cand = candidates[uidx]
                probs = []
                for n in cand:
                    eta = compute_eta(uidx, n, remC, remBu, remBd)
                    col = n if n!=-1 else N
                    probs.append((pher[uidx,col]**alpha)*(eta**beta))
                probs = np.array(probs)
                probs = probs/probs.sum() if probs.sum()>0 else None
                choice = np.random.choice(cand, p=probs) if probs is not None else -1
                # greedy feasibility check
                for n in sorted(cand, key=lambda c: -probs[cand.index(c)] if probs is not None else 0):
                    if n==-1: assignment[uidx]=-1; break
                    s=users[uidx]['s']
                    if remC[n]>=services[s]['cs'] and remBu[n]>=services[s]['bu'] and remBd[n]>=services[s]['bd']:
                        assignment[uidx]=n; remC[n]-=services[s]['cs']; remBu[n]-=services[s]['bu']; remBd[n]-=services[s]['bd']; edge_served+=1; break
            if edge_served>iter_best_score:
                iter_best_score, iter_best_assign = edge_served, assignment
            for uidx,a in enumerate(assignment):
                col = a if a!=-1 else N
                delta_pher[uidx,col]+=edge_served/(U+1e-9)
        pher=(1-rho)*pher+rho*(delta_pher/(delta_pher.max()+1e-9))
        if iter_best_score>best_global_score:
            best_global_score,best_global=iter_best_score,iter_best_assign

    return best_global, best_global_score, {}

# ---------------------------
# GA core
# ---------------------------
def evaluate_chromosome(vec):
    placement = vector_to_placement(np.array(vec))
    repair_placement(placement)
    _, edge_served, usage = aco_routing(placement)
    return edge_served, placement, usage

def tournament_selection(pop, fitnesses, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[max(idxs, key=lambda i: fitnesses[i])]

def uniform_crossover(p1,p2):
    mask = np.random.rand(len(p1))<0.5
    child=p1.copy(); child[mask]=p2[mask]; return child

def mutate_vector(vec,rate=MUTATION_RATE):
    for i in range(len(vec)):
        if random.random()<rate: vec[i]=1-vec[i]
    return vec

def run_hybrid_ga_aco():
    population=[placement_to_vector(rand_feasible_placement()) for _ in range(POP_SIZE)]
    best_vec,best_fit=None,-1
    for gen in range(GENERATIONS):
        fitnesses=[]
        for indiv in population:
            fit,_,_=evaluate_chromosome(indiv)
            fitnesses.append(fit)
            if fit>best_fit: best_fit, best_vec=fit, indiv.copy()
        if gen%3==0 or gen==GENERATIONS-1:
            print(f"[Gen {gen}] best_fit={max(fitnesses)} mean_fit={np.mean(fitnesses):.2f}")
        ranked=sorted(list(zip(population,fitnesses)),key=lambda x:x[1],reverse=True)
        elites=[deepcopy(x[0]) for x in ranked[:ELITISM]]
        new_pop=elites.copy()
        while len(new_pop)<POP_SIZE:
            p1=tournament_selection(population,fitnesses)
            p2=tournament_selection(population,fitnesses)
            child=p1.copy()
            if random.random()<CROSSOVER_RATE: child=uniform_crossover(p1,p2)
            child=mutate_vector(child,MUTATION_RATE)
            child=placement_to_vector(repair_placement(vector_to_placement(child)))
            new_pop.append(child)
        population=new_pop
    return best_vec,best_fit

# ---------------------------
# Run
# ---------------------------
best_vec,best_fit=run_hybrid_ga_aco()
print("\nFinal best edge-served:",best_fit,"/",U)

[Gen 0] best_fit=33 mean_fit=27.00
[Gen 3] best_fit=36 mean_fit=32.83
[Gen 6] best_fit=37 mean_fit=34.92
[Gen 9] best_fit=37 mean_fit=35.00
[Gen 11] best_fit=37 mean_fit=36.08

Final best edge-served: 37 / 100


In [2]:
import random, time, numpy as np
from collections import defaultdict
from copy import deepcopy

# ---------------------------
# Parameters
# ---------------------------
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

N = 3           # number of base stations
S = 16          # number of services
U = 100         # number of users

# BS resource ranges
STORAGE_RANGE = (200, 400)   # GB
CPU_RANGE     = (2.0, 6.0)   # GHz
UPLINK_RANGE  = (30, 80)     # Mbps
DOWNLINK_RANGE= (60, 150)    # Mbps

# Service requirement ranges
RS_RANGE = (8, 40)        # GB
CS_RANGE = (0.05, 0.5)    # GHz
BU_RANGE = (0.5, 4)       # Mbps
BD_RANGE = (0.5, 12)      # Mbps

# User coverage
MIN_COV = 1
MAX_COV = 3

# Popularity skew (Zipf)
ZIPF_SKEW = 0.9

# GA hyperparameters
POP_SIZE = 12
GENERATIONS = 12
CROSSOVER_RATE = 0.8
MUTATION_RATE  = 0.04
ELITISM = 2

# ACO hyperparameters
ANTS = 6
ACO_ITERS = 6
ALPHA = 1.0
BETA  = 2.0
RHO   = 0.15

# ---------------------------
# Build synthetic scenario
# ---------------------------
BS = []
for n in range(N):
    BS.append({
        'R': random.randint(*STORAGE_RANGE),
        'C': random.uniform(*CPU_RANGE),
        'Bu': random.randint(*UPLINK_RANGE),
        'Bd': random.randint(*DOWNLINK_RANGE),
    })

ranks = np.arange(1, S + 1)
zipf_weights = 1.0 / (ranks ** ZIPF_SKEW)
zipf_probs   = zipf_weights / zipf_weights.sum()

services = []
for s in range(S):
    services.append({
        'rs': random.randint(*RS_RANGE),
        'cs': random.uniform(*CS_RANGE),
        'bu': random.uniform(*BU_RANGE),
        'bd': random.uniform(*BD_RANGE),
        'pop_prob': float(zipf_probs[s])
    })

users = []
service_choices = list(range(S))
for u in range(U):
    su = int(np.random.choice(service_choices, p=zipf_probs))
    cov_size = random.randint(MIN_COV, min(MAX_COV, N))
    nearby = sorted(random.sample(range(N), cov_size))
    users.append({'s': su, 'nearby': nearby})

users_by_service = defaultdict(list)
for uid, u in enumerate(users):
    users_by_service[u['s']].append(uid)

# ---------------------------
# Helpers
# ---------------------------
def rand_feasible_placement():
    x = np.zeros((N, S), dtype=int)
    for n in range(N):
        perm = list(range(S))
        random.shuffle(perm)
        used = 0
        for s in perm:
            rs = services[s]['rs']
            if used + rs <= BS[n]['R']:
                if random.random() < 0.35:
                    x[n, s] = 1
                    used += rs
    repair_placement(x)
    return x

def placement_to_vector(x):
    return x.flatten()

def vector_to_placement(v):
    return v.reshape((N, S)).astype(int)

def compute_marginal_benefit_per_gb(bs_index, placement):
    benefit = {}
    for s in range(S):
        if placement[bs_index, s] == 0: continue
        count = 0.0
        for uid in users_by_service[s]:
            if bs_index in users[uid]['nearby']:
                other_hosts = [m for m in users[uid]['nearby']
                               if m != bs_index and placement[m, s] == 1]
                count += 1.0 if len(other_hosts)==0 else 0.2
        rs = services[s]['rs']
        benefit[s] = count / (rs + 1e-9)
    return benefit

def repair_placement(placement):
    for n in range(N):
        used = int(placement[n] @ np.array([services[s]['rs'] for s in range(S)]))
        if used <= BS[n]['R']: continue
        benefit = compute_marginal_benefit_per_gb(n, placement)
        placed = [s for s in range(S) if placement[n, s]==1]
        placed_sorted = sorted(placed, key=lambda s: benefit.get(s, 0.0))
        idx = 0
        while used > BS[n]['R'] and idx < len(placed_sorted):
            srm = placed_sorted[idx]
            placement[n, srm] = 0
            used -= services[srm]['rs']
            idx += 1
        if used > BS[n]['R']:
            placed = [s for s in range(S) if placement[n, s]==1]
            random.shuffle(placed)
            for srm in placed:
                placement[n, srm] = 0
                used -= services[srm]['rs']
                if used <= BS[n]['R']:
                    break
    return placement

# ---------------------------
# ACO routing
# ---------------------------
def aco_routing(placement, ants=ANTS, iterations=ACO_ITERS, alpha=ALPHA, beta=BETA, rho=RHO):
    candidates = []
    for u in users:
        su = u['s']
        cand = [n for n in u['nearby'] if placement[n, su]==1]
        candidates.append(cand + [-1] if cand else [-1])
    pher = np.ones((U, N+1)) * (1.0/(N+1))
    best_global, best_global_score = None, -1

    def compute_eta(u_idx, n_idx, remC, remBu, remBd):
        if n_idx == -1: return 0.1
        s = users[u_idx]['s']
        cs, bu, bd = services[s]['cs'], services[s]['bu'], services[s]['bd']
        return max(min(remC[n_idx]/cs, remBu[n_idx]/bu, remBd[n_idx]/bd), 1e-6)

    for _ in range(iterations):
        iter_best_score, iter_best_assign = -1, None
        delta_pher = np.zeros_like(pher)
        for _ in range(ants):
            remC = np.array([BS[n]['C'] for n in range(N)])
            remBu= np.array([BS[n]['Bu'] for n in range(N)])
            remBd= np.array([BS[n]['Bd'] for n in range(N)])
            assignment = [-1]*U; edge_served=0
            for uidx in range(U):
                cand = candidates[uidx]
                probs = []
                for n in cand:
                    eta = compute_eta(uidx, n, remC, remBu, remBd)
                    col = n if n!=-1 else N
                    probs.append((pher[uidx,col]**alpha)*(eta**beta))
                probs = np.array(probs)
                probs = probs/probs.sum() if probs.sum()>0 else None
                choice = np.random.choice(cand, p=probs) if probs is not None else -1
                # greedy feasibility check
                for n in sorted(cand, key=lambda c: -probs[cand.index(c)] if probs is not None else 0):
                    if n==-1: assignment[uidx]=-1; break
                    s=users[uidx]['s']
                    if remC[n]>=services[s]['cs'] and remBu[n]>=services[s]['bu'] and remBd[n]>=services[s]['bd']:
                        assignment[uidx]=n; remC[n]-=services[s]['cs']; remBu[n]-=services[s]['bu']; remBd[n]-=services[s]['bd']; edge_served+=1; break
            if edge_served>iter_best_score:
                iter_best_score, iter_best_assign = edge_served, assignment
            for uidx,a in enumerate(assignment):
                col = a if a!=-1 else N
                delta_pher[uidx,col]+=edge_served/(U+1e-9)
        pher=(1-rho)*pher+rho*(delta_pher/(delta_pher.max()+1e-9))
        if iter_best_score>best_global_score:
            best_global_score,best_global=iter_best_score,iter_best_assign

    # usage summary
    usage = {'cpu_util':[0]*N,'uplink_util':[0]*N,'downlink_util':[0]*N}
    for uidx,a in enumerate(best_global):
        if a==-1: continue
        s=users[uidx]['s']
        usage['cpu_util'][a]+=services[s]['cs']
        usage['uplink_util'][a]+=services[s]['bu']
        usage['downlink_util'][a]+=services[s]['bd']
    return best_global, best_global_score, usage

# ---------------------------
# GA core
# ---------------------------
def evaluate_chromosome(vec):
    placement = vector_to_placement(np.array(vec))
    repair_placement(placement)
    _, edge_served, usage = aco_routing(placement)
    return edge_served, placement, usage

def tournament_selection(pop, fitnesses, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[max(idxs, key=lambda i: fitnesses[i])]

def uniform_crossover(p1,p2):
    mask = np.random.rand(len(p1))<0.5
    child=p1.copy(); child[mask]=p2[mask]; return child

def mutate_vector(vec,rate=MUTATION_RATE):
    for i in range(len(vec)):
        if random.random()<rate: vec[i]=1-vec[i]
    return vec

def run_hybrid_ga_aco():
    population=[placement_to_vector(rand_feasible_placement()) for _ in range(POP_SIZE)]
    best_vec,best_fit=None,-1
    for gen in range(GENERATIONS):
        fitnesses=[]
        for indiv in population:
            fit,_,_=evaluate_chromosome(indiv)
            fitnesses.append(fit)
            if fit>best_fit: best_fit, best_vec=fit, indiv.copy()
        if gen%3==0 or gen==GENERATIONS-1:
            print(f"[Gen {gen}] best_fit={max(fitnesses)} mean_fit={np.mean(fitnesses):.2f}")
        ranked=sorted(list(zip(population,fitnesses)),key=lambda x:x[1],reverse=True)
        elites=[deepcopy(x[0]) for x in ranked[:ELITISM]]
        new_pop=elites.copy()
        while len(new_pop)<POP_SIZE:
            p1=tournament_selection(population,fitnesses)
            p2=tournament_selection(population,fitnesses)
            child=p1.copy()
            if random.random()<CROSSOVER_RATE: child=uniform_crossover(p1,p2)
            child=mutate_vector(child,MUTATION_RATE)
            child=placement_to_vector(repair_placement(vector_to_placement(child)))
            new_pop.append(child)
        population=new_pop
    return best_vec,best_fit

# ---------------------------
# Run
# ---------------------------
best_vec,best_fit=run_hybrid_ga_aco()

# Final evaluation
best_placement = vector_to_placement(np.array(best_vec))
best_assignment, best_edge_served, usage = aco_routing(best_placement)

accuracy   = best_edge_served / U
hit_ratio  = accuracy
miss_ratio = 1 - hit_ratio

print("\n===== Final Detailed Results =====")
print(f"Total Users: {U}")
print(f"Edge-served Users: {best_edge_served}")
print(f"Cloud-served Users: {U - best_edge_served}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Hit Ratio: {hit_ratio*100:.2f}%")
print(f"Miss Ratio: {miss_ratio*100:.2f}%\n")

# Per-BS usage summary
print("Per-BS resource utilizations:")
for n in range(N):
    placed_services = [s for s in range(S) if best_placement[n, s] == 1]
    storage_used = sum(services[s]['rs'] for s in placed_services)
    storage_util = storage_used / BS[n]['R'] * 100
    print(f"BS{n}: services={placed_services}")
    print(f"     storage_used={storage_used}/{BS[n]['R']}GB ({storage_util:.1f}%)")
    print(f"     cpu_used≈{usage.get('cpu_util',[0]*N)[n]:.2f} GHz")
    print(f"     uplink_used≈{usage.get('uplink_util',[0]*N)[n]:.2f} Mbps")
    print(f"     downlink_used≈{usage.get('downlink_util',[0]*N)[n]:.2f} Mbps\n")

[Gen 0] best_fit=33 mean_fit=27.00
[Gen 3] best_fit=36 mean_fit=32.83
[Gen 6] best_fit=37 mean_fit=34.92
[Gen 9] best_fit=37 mean_fit=35.00
[Gen 11] best_fit=37 mean_fit=36.08

===== Final Detailed Results =====
Total Users: 100
Edge-served Users: 37
Cloud-served Users: 63
Accuracy: 37.00%
Hit Ratio: 37.00%
Miss Ratio: 63.00%

Per-BS resource utilizations:
BS0: services=[0, 2, 10, 13, 14]
     storage_used=97/234GB (41.5%)
     cpu_used≈4.23 GHz
     uplink_used≈47.56 Mbps
     downlink_used≈63.12 Mbps

BS1: services=[0, 4, 5, 11]
     storage_used=128/265GB (48.3%)
     cpu_used≈2.43 GHz
     uplink_used≈26.61 Mbps
     downlink_used≈56.83 Mbps

BS2: services=[1, 2, 4, 10, 11, 12, 14, 15]
     storage_used=177/320GB (55.3%)
     cpu_used≈4.59 GHz
     uplink_used≈19.53 Mbps
     downlink_used≈80.95 Mbps



In [4]:
import random, time, numpy as np, pandas as pd
from collections import defaultdict
from copy import deepcopy

# ---------------------------
# Parameters
# ---------------------------
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

POP_SIZE = 12
GENERATIONS = 12
CROSSOVER_RATE = 0.8
MUTATION_RATE  = 0.04
ELITISM = 2

ANTS = 6
ACO_ITERS = 6
ALPHA = 1.0
BETA  = 2.0
RHO   = 0.15

# ---------------------------
# Load Data
# ---------------------------
BS_df = pd.read_csv("bs.csv")
SERV_df = pd.read_csv("services.csv")
USERS_df = pd.read_csv("users.csv")

N = len(BS_df)
S = len(SERV_df)
U = len(USERS_df)

BS = []
for _, row in BS_df.iterrows():
    BS.append({
        'R': row['storage'],
        'C': row['cpu'],
        'Bu': row['uplink'],
        'Bd': row['downlink'],
    })

services = []
for _, row in SERV_df.iterrows():
    services.append({
        'rs': row['storage_req'],
        'cs': row['cpu_req'],
        'bu': row['uplink_req'],
        'bd': row['downlink_req'],
        'pop_prob': row['pop_prob']
    })

users = []
for _, row in USERS_df.iterrows():
    nearby = list(map(int, str(row['nearby_bs']).split()))
    users.append({'s': int(row['service']), 'nearby': nearby})

users_by_service = defaultdict(list)
for uid, u in enumerate(users):
    users_by_service[u['s']].append(uid)

# ---------------------------
# Helpers
# ---------------------------
def rand_feasible_placement():
    x = np.zeros((N, S), dtype=int)
    for n in range(N):
        perm = list(range(S))
        random.shuffle(perm)
        used = 0
        for s in perm:
            rs = services[s]['rs']
            if used + rs <= BS[n]['R']:
                if random.random() < 0.35:
                    x[n, s] = 1
                    used += rs
    repair_placement(x)
    return x

def placement_to_vector(x): return x.flatten()
def vector_to_placement(v): return v.reshape((N, S)).astype(int)

def compute_marginal_benefit_per_gb(bs_index, placement):
    benefit = {}
    for s in range(S):
        if placement[bs_index, s] == 0: continue
        count = 0.0
        for uid in users_by_service[s]:
            if bs_index in users[uid]['nearby']:
                other_hosts = [m for m in users[uid]['nearby']
                               if m != bs_index and placement[m, s] == 1]
                count += 1.0 if len(other_hosts)==0 else 0.2
        rs = services[s]['rs']
        benefit[s] = count / (rs + 1e-9)
    return benefit

def repair_placement(placement):
    for n in range(N):
        used = int(placement[n] @ np.array([services[s]['rs'] for s in range(S)]))
        if used <= BS[n]['R']: continue
        benefit = compute_marginal_benefit_per_gb(n, placement)
        placed = [s for s in range(S) if placement[n, s]==1]
        placed_sorted = sorted(placed, key=lambda s: benefit.get(s, 0.0))
        idx = 0
        while used > BS[n]['R'] and idx < len(placed_sorted):
            srm = placed_sorted[idx]
            placement[n, srm] = 0
            used -= services[srm]['rs']
            idx += 1
    return placement

# ---------------------------
# ACO routing
# ---------------------------
def aco_routing(placement, ants=ANTS, iterations=ACO_ITERS, alpha=ALPHA, beta=BETA, rho=RHO):
    candidates = []
    for u in users:
        su = u['s']
        cand = [n for n in u['nearby'] if placement[n, su]==1]
        candidates.append(cand + [-1] if cand else [-1])
    pher = np.ones((U, N+1)) * (1.0/(N+1))
    best_global, best_global_score = None, -1

    def compute_eta(u_idx, n_idx, remC, remBu, remBd):
        if n_idx == -1: return 0.1
        s = users[u_idx]['s']
        cs, bu, bd = services[s]['cs'], services[s]['bu'], services[s]['bd']
        return max(min(remC[n_idx]/cs, remBu[n_idx]/bu, remBd[n_idx]/bd), 1e-6)

    for _ in range(iterations):
        iter_best_score, iter_best_assign = -1, None
        delta_pher = np.zeros_like(pher)
        for _ in range(ants):
            remC = np.array([BS[n]['C'] for n in range(N)])
            remBu= np.array([BS[n]['Bu'] for n in range(N)])
            remBd= np.array([BS[n]['Bd'] for n in range(N)])
            assignment = [-1]*U; edge_served=0
            for uidx in range(U):
                cand = candidates[uidx]
                probs = []
                for n in cand:
                    eta = compute_eta(uidx, n, remC, remBu, remBd)
                    col = n if n!=-1 else N
                    probs.append((pher[uidx,col]**alpha)*(eta**beta))
                probs = np.array(probs)
                probs = probs/probs.sum() if probs.sum()>0 else None
                # greedy feasibility
                for n in sorted(cand, key=lambda c: -probs[cand.index(c)] if probs is not None else 0):
                    if n==-1: assignment[uidx]=-1; break
                    s=users[uidx]['s']
                    if remC[n]>=services[s]['cs'] and remBu[n]>=services[s]['bu'] and remBd[n]>=services[s]['bd']:
                        assignment[uidx]=n
                        remC[n]-=services[s]['cs']
                        remBu[n]-=services[s]['bu']
                        remBd[n]-=services[s]['bd']
                        edge_served+=1
                        break
            if edge_served>iter_best_score:
                iter_best_score, iter_best_assign = edge_served, assignment
            for uidx,a in enumerate(assignment):
                col = a if a!=-1 else N
                delta_pher[uidx,col]+=edge_served/(U+1e-9)
        pher=(1-rho)*pher+rho*(delta_pher/(delta_pher.max()+1e-9))
        if iter_best_score>best_global_score:
            best_global_score,best_global=iter_best_score,iter_best_assign

    usage = {'cpu_util':[0]*N,'uplink_util':[0]*N,'downlink_util':[0]*N}
    for uidx,a in enumerate(best_global):
        if a==-1: continue
        s=users[uidx]['s']
        usage['cpu_util'][a]+=services[s]['cs']
        usage['uplink_util'][a]+=services[s]['bu']
        usage['downlink_util'][a]+=services[s]['bd']
    return best_global, best_global_score, usage

# ---------------------------
# GA core
# ---------------------------
def evaluate_chromosome(vec):
    placement = vector_to_placement(np.array(vec))
    repair_placement(placement)
    _, edge_served, usage = aco_routing(placement)
    return edge_served, placement, usage

def tournament_selection(pop, fitnesses, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[max(idxs, key=lambda i: fitnesses[i])]

def uniform_crossover(p1,p2):
    mask = np.random.rand(len(p1))<0.5
    child=p1.copy(); child[mask]=p2[mask]; return child

def mutate_vector(vec,rate=MUTATION_RATE):
    for i in range(len(vec)):
        if random.random()<rate: vec[i]=1-vec[i]
    return vec

def run_hybrid_ga_aco():
    population=[placement_to_vector(rand_feasible_placement()) for _ in range(POP_SIZE)]
    best_vec,best_fit=None,-1
    for gen in range(GENERATIONS):
        fitnesses=[]
        for indiv in population:
            fit,_,_=evaluate_chromosome(indiv)
            fitnesses.append(fit)
            if fit>best_fit: best_fit, best_vec=fit, indiv.copy()
        if gen%3==0 or gen==GENERATIONS-1:
            print(f"[Gen {gen}] best_fit={max(fitnesses)} mean_fit={np.mean(fitnesses):.2f}")
        ranked=sorted(list(zip(population,fitnesses)),key=lambda x:x[1],reverse=True)
        elites=[deepcopy(x[0]) for x in ranked[:ELITISM]]
        new_pop=elites.copy()
        while len(new_pop)<POP_SIZE:
            p1=tournament_selection(population,fitnesses)
            p2=tournament_selection(population,fitnesses)
            child=p1.copy()
            if random.random()<CROSSOVER_RATE: child=uniform_crossover(p1,p2)
            child=mutate_vector(child,MUTATION_RATE)
            child=placement_to_vector(repair_placement(vector_to_placement(child)))
            new_pop.append(child)
        population=new_pop
    return best_vec,best_fit

# ---------------------------
# Run
# ---------------------------
best_vec,best_fit=run_hybrid_ga_aco()

best_placement = vector_to_placement(np.array(best_vec))
best_assignment, best_edge_served, usage = aco_routing(best_placement)

accuracy   = best_edge_served / U
hit_ratio  = accuracy
miss_ratio = 1 - hit_ratio

print("\n===== Final Detailed Results =====")
print(f"Total Users: {U}")
print(f"Edge-served Users: {best_edge_served}")
print(f"Cloud-served Users: {U - best_edge_served}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Hit Ratio: {hit_ratio*100:.2f}%")
print(f"Miss Ratio: {miss_ratio*100:.2f}%\n")

print("Per-BS resource utilizations:")
for n in range(N):
    placed_services = [s for s in range(S) if best_placement[n, s] == 1]
    storage_used = sum(services[s]['rs'] for s in placed_services)
    storage_util = storage_used / BS[n]['R'] * 100
    print(f"BS{n}: services={placed_services}")
    print(f"     storage_used={storage_used}/{BS[n]['R']}GB ({storage_util:.1f}%)")
    print(f"     cpu_used≈{usage['cpu_util'][n]:.2f} GHz")
    print(f"     uplink_used≈{usage['uplink_util'][n]:.2f} Mbps")
    print(f"     downlink_used≈{usage['downlink_util'][n]:.2f} Mbps\n")

FileNotFoundError: [Errno 2] No such file or directory: 'bs.csv'

In [5]:
# ===============================
# Step 1: Upload datasets.zip
# ===============================
from google.colab import files
import zipfile, os, random, time, numpy as np, pandas as pd
from collections import defaultdict
from copy import deepcopy

uploaded = files.upload()   # choose datasets.zip

# ===============================
# Step 2: Extract ZIP
# ===============================
with zipfile.ZipFile("datasets.zip", "r") as zip_ref:
    zip_ref.extractall("datasets")

print("Extracted files:", os.listdir("datasets"))

# ===============================
# Step 3: Parameters
# ===============================
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

POP_SIZE = 12
GENERATIONS = 12
CROSSOVER_RATE = 0.8
MUTATION_RATE  = 0.04
ELITISM = 2

ANTS = 6
ACO_ITERS = 6
ALPHA = 1.0
BETA  = 2.0
RHO   = 0.15

# ===============================
# Step 4: Load Data
# ===============================
BS_df = pd.read_csv("datasets/bs.csv")
SERV_df = pd.read_csv("datasets/services.csv")
USERS_df = pd.read_csv("datasets/users.csv")

N = len(BS_df)
S = len(SERV_df)
U = len(USERS_df)

BS = []
for _, row in BS_df.iterrows():
    BS.append({
        'R': row['storage'],
        'C': row['cpu'],
        'Bu': row['uplink'],
        'Bd': row['downlink'],
    })

services = []
for _, row in SERV_df.iterrows():
    services.append({
        'rs': row['storage_req'],
        'cs': row['cpu_req'],
        'bu': row['uplink_req'],
        'bd': row['downlink_req'],
        'pop_prob': row['pop_prob']
    })

users = []
for _, row in USERS_df.iterrows():
    nearby = list(map(int, str(row['nearby_bs']).split()))
    users.append({'s': int(row['service']), 'nearby': nearby})

users_by_service = defaultdict(list)
for uid, u in enumerate(users):
    users_by_service[u['s']].append(uid)

# ===============================
# Step 5: Helper Functions
# ===============================
def rand_feasible_placement():
    x = np.zeros((N, S), dtype=int)
    for n in range(N):
        perm = list(range(S))
        random.shuffle(perm)
        used = 0
        for s in perm:
            rs = services[s]['rs']
            if used + rs <= BS[n]['R']:
                if random.random() < 0.35:
                    x[n, s] = 1
                    used += rs
    repair_placement(x)
    return x

def placement_to_vector(x): return x.flatten()
def vector_to_placement(v): return v.reshape((N, S)).astype(int)

def compute_marginal_benefit_per_gb(bs_index, placement):
    benefit = {}
    for s in range(S):
        if placement[bs_index, s] == 0: continue
        count = 0.0
        for uid in users_by_service[s]:
            if bs_index in users[uid]['nearby']:
                other_hosts = [m for m in users[uid]['nearby']
                               if m != bs_index and placement[m, s] == 1]
                count += 1.0 if len(other_hosts)==0 else 0.2
        rs = services[s]['rs']
        benefit[s] = count / (rs + 1e-9)
    return benefit

def repair_placement(placement):
    for n in range(N):
        used = int(placement[n] @ np.array([services[s]['rs'] for s in range(S)]))
        if used <= BS[n]['R']: continue
        benefit = compute_marginal_benefit_per_gb(n, placement)
        placed = [s for s in range(S) if placement[n, s]==1]
        placed_sorted = sorted(placed, key=lambda s: benefit.get(s, 0.0))
        idx = 0
        while used > BS[n]['R'] and idx < len(placed_sorted):
            srm = placed_sorted[idx]
            placement[n, srm] = 0
            used -= services[srm]['rs']
            idx += 1
    return placement

# ===============================
# Step 6: ACO Routing
# ===============================
def aco_routing(placement, ants=ANTS, iterations=ACO_ITERS, alpha=ALPHA, beta=BETA, rho=RHO):
    candidates = []
    for u in users:
        su = u['s']
        cand = [n for n in u['nearby'] if placement[n, su]==1]
        candidates.append(cand + [-1] if cand else [-1])
    pher = np.ones((U, N+1)) * (1.0/(N+1))
    best_global, best_global_score = None, -1

    def compute_eta(u_idx, n_idx, remC, remBu, remBd):
        if n_idx == -1: return 0.1
        s = users[u_idx]['s']
        cs, bu, bd = services[s]['cs'], services[s]['bu'], services[s]['bd']
        return max(min(remC[n_idx]/cs, remBu[n_idx]/bu, remBd[n_idx]/bd), 1e-6)

    for _ in range(iterations):
        iter_best_score, iter_best_assign = -1, None
        delta_pher = np.zeros_like(pher)
        for _ in range(ants):
            remC = np.array([BS[n]['C'] for n in range(N)])
            remBu= np.array([BS[n]['Bu'] for n in range(N)])
            remBd= np.array([BS[n]['Bd'] for n in range(N)])
            assignment = [-1]*U; edge_served=0
            for uidx in range(U):
                cand = candidates[uidx]
                probs = []
                for n in cand:
                    eta = compute_eta(uidx, n, remC, remBu, remBd)
                    col = n if n!=-1 else N
                    probs.append((pher[uidx,col]**alpha)*(eta**beta))
                probs = np.array(probs)
                probs = probs/probs.sum() if probs.sum()>0 else None
                for n in sorted(cand, key=lambda c: -probs[cand.index(c)] if probs is not None else 0):
                    if n==-1: assignment[uidx]=-1; break
                    s=users[uidx]['s']
                    if remC[n]>=services[s]['cs'] and remBu[n]>=services[s]['bu'] and remBd[n]>=services[s]['bd']:
                        assignment[uidx]=n
                        remC[n]-=services[s]['cs']
                        remBu[n]-=services[s]['bu']
                        remBd[n]-=services[s]['bd']
                        edge_served+=1
                        break
            if edge_served>iter_best_score:
                iter_best_score, iter_best_assign = edge_served, assignment
            for uidx,a in enumerate(assignment):
                col = a if a!=-1 else N
                delta_pher[uidx,col]+=edge_served/(U+1e-9)
        pher=(1-rho)*pher+rho*(delta_pher/(delta_pher.max()+1e-9))
        if iter_best_score>best_global_score:
            best_global_score,best_global=iter_best_score,iter_best_assign

    usage = {'cpu_util':[0]*N,'uplink_util':[0]*N,'downlink_util':[0]*N}
    for uidx,a in enumerate(best_global):
        if a==-1: continue
        s=users[uidx]['s']
        usage['cpu_util'][a]+=services[s]['cs']
        usage['uplink_util'][a]+=services[s]['bu']
        usage['downlink_util'][a]+=services[s]['bd']
    return best_global, best_global_score, usage

# ===============================
# Step 7: GA Core
# ===============================
def evaluate_chromosome(vec):
    placement = vector_to_placement(np.array(vec))
    repair_placement(placement)
    _, edge_served, usage = aco_routing(placement)
    return edge_served, placement, usage

def tournament_selection(pop, fitnesses, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[max(idxs, key=lambda i: fitnesses[i])]

def uniform_crossover(p1,p2):
    mask = np.random.rand(len(p1))<0.5
    child=p1.copy(); child[mask]=p2[mask]; return child

def mutate_vector(vec,rate=MUTATION_RATE):
    for i in range(len(vec)):
        if random.random()<rate: vec[i]=1-vec[i]
    return vec

def run_hybrid_ga_aco():
    population=[placement_to_vector(rand_feasible_placement()) for _ in range(POP_SIZE)]
    best_vec,best_fit=None,-1
    for gen in range(GENERATIONS):
        fitnesses=[]
        for indiv in population:
            fit,_,_=evaluate_chromosome(indiv)
            fitnesses.append(fit)
            if fit>best_fit: best_fit, best_vec=fit, indiv.copy()
        if gen%3==0 or gen==GENERATIONS-1:
            print(f"[Gen {gen}] best_fit={max(fitnesses)} mean_fit={np.mean(fitnesses):.2f}")
        ranked=sorted(list(zip(population,fitnesses)),key=lambda x:x[1],reverse=True)
        elites=[deepcopy(x[0]) for x in ranked[:ELITISM]]
        new_pop=elites.copy()
        while len(new_pop)<POP_SIZE:
            p1=tournament_selection(population,fitnesses)
            p2=tournament_selection(population,fitnesses)
            child=p1.copy()
            if random.random()<CROSSOVER_RATE: child=uniform_crossover(p1,p2)
            child=mutate_vector(child,MUTATION_RATE)
            child=placement_to_vector(repair_placement(vector_to_placement(child)))
            new_pop.append(child)
        population=new_pop
    return best_vec,best_fit

# ===============================
# Step 8: Run
# ===============================
best_vec,best_fit=run_hybrid_ga_aco()

best_placement = vector_to_placement(np.array(best_vec))
best_assignment, best_edge_served, usage = aco_routing(best_placement)

accuracy   = best_edge_served / U
hit_ratio  = accuracy
miss_ratio = 1 - hit_ratio

print("\n===== Final Detailed Results =====")
print(f"Total Users: {U}")
print(f"Edge-served Users: {best_edge_served}")
print(f"Cloud-served Users: {U - best_edge_served}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Hit Ratio: {hit_ratio*100:.2f}%")
print(f"Miss Ratio: {miss_ratio*100:.2f}%\n")

print("Per-BS resource utilizations:")
for n in range(N):
    placed_services = [s for s in range(S) if best_placement[n, s] == 1]
    storage_used = sum(services[s]['rs'] for s in placed_services)
    storage_util = storage_used / BS[n]['R'] * 100
    print(f"BS{n}: services={placed_services}")
    print(f"     storage_used={storage_used}/{BS[n]['R']}GB ({storage_util:.1f}%)")
    print(f"     cpu_used≈{usage['cpu_util'][n]:.2f} GHz")
    print(f"     uplink_used≈{usage['uplink_util'][n]:.2f} Mbps")
    print(f"     downlink_used≈{usage['downlink_util'][n]:.2f} Mbps\n")

Saving datasets.zip to datasets (1).zip
Extracted files: ['users.csv', 'services.csv', 'bs.csv']
[Gen 0] best_fit=25 mean_fit=16.08
[Gen 3] best_fit=26 mean_fit=24.50
[Gen 6] best_fit=29 mean_fit=26.58
[Gen 9] best_fit=30 mean_fit=28.50
[Gen 11] best_fit=30 mean_fit=29.25

===== Final Detailed Results =====
Total Users: 30
Edge-served Users: 30
Cloud-served Users: 0
Accuracy: 100.00%
Hit Ratio: 100.00%
Miss Ratio: 0.00%

Per-BS resource utilizations:
BS0: services=[0, 1, 2, 3, 5, 6, 7, 8, 9]
     storage_used=190.0/300.0GB (63.3%)
     cpu_used≈2.77 GHz
     uplink_used≈23.50 Mbps
     downlink_used≈61.00 Mbps

BS1: services=[1, 2, 4, 6, 7, 8, 9]
     storage_used=167.0/250.0GB (66.8%)
     cpu_used≈1.40 GHz
     uplink_used≈11.50 Mbps
     downlink_used≈29.00 Mbps

BS2: services=[0, 1, 3, 4, 5, 6, 7, 8, 9]
     storage_used=180.0/320.0GB (56.2%)
     cpu_used≈2.79 GHz
     uplink_used≈28.00 Mbps
     downlink_used≈66.00 Mbps

